In [1]:
!pip install --no-deps unsloth vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 4.4 MB/s eta 0:00:00


In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [ ]:
from google.colab import userdata
userdata.get('huging_face_token')

In [4]:
from unsloth import FastLanguageModel
import torch # Import PyTorch
from trl import SFTTrainer # Trainer for supervised fine-tuning (SFT)
from unsloth import is_bfloat16_supported # Checks if the hardware supports bfloat16 precision
# Hugging Face modules
from huggingface_hub import login # Lets you login to API
from transformers import TrainingArguments # Defines training hyperparameters
from datasets import load_dataset # Lets you load fine-tuning datasets
# Import weights and biases
import wandb
# Import kaggle secrets
# from kaggle_secrets import UserSecretsClient

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-04 17:06:32 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-04 17:06:32 [__init__.py:239] Automatically detected platform cuda.


In [5]:
hugging_face_token = userdata.get('huging_face_token')


In [8]:
from unsloth import FastModel
import torch



In [9]:
import torch
torch.cuda.empty_cache()

model, tokenizer = FastModel.from_pretrained(
    model_name="unsloth/gemma-3-4b-it",
    # token=hugging_face_token, # Use hugging face token
    max_seq_length= 2048, # Ensure the model can process up to 2048 tokens at once
    load_in_4bit=True,
    load_in_8bit=False,
    full_finetuning=False,
    #dtype=torch.bfloat16, # COMMENTED OUT to use fp16 instead of bfloat16
    # token=hugging_face_token,
)

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name="google/gemma-3-4b",
#     max_seq_length=max_seq_length,
#     dtype="bf16",  # or "fp16"
#     load_in_4bit=False,
#     token=hugging_face_token,
# )


==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [10]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layer = False,
    finetune_language_layer = True,
    finetune_attention_modules=True,
    finetune_mlp_modules=True,
    r = 8,
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,

                                 )

Unsloth: Making `base_model.model.vision_tower.vision_model` require gradients


In [11]:
from unsloth.chat_templates import get_chat_template, standardize_data_formats
tokenizer = get_chat_template(
    tokenizer,
    chat_template = 'gemma-3'
)

In [12]:
import json

data = []
with open('/content/Dataset.jsonl', 'r') as jsonl_file:
    for line in jsonl_file:
        data.append(json.loads(line))

In [13]:
dataset = standardize_data_formats(data)

from datasets import Dataset
# Step 3: Convert to HuggingFace Dataset
hf_dataset = Dataset.from_list(dataset)

In [14]:

# Step 4: Set up tokenizer with chat template
tokenizer = get_chat_template(tokenizer, chat_template='gemma-3')


In [15]:
def is_valid_conversation(example):
    messages = example["messages"]
    if not messages or messages[0]["role"] != "user":
        return False
    for i in range(1, len(messages)):
        if messages[i]["role"] == messages[i - 1]["role"]:
            return False
    return True

# Filter dataset before applying template
hf_dataset = hf_dataset.filter(is_valid_conversation)


Filter:   0%|          | 0/136 [00:00<?, ? examples/s]

In [16]:

# Step 5: Apply chat template to each conversation
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples['messages'])  # ← Fix here
    return {'text': texts}


# Step 6: Map and apply template
hf_dataset = hf_dataset.map(apply_chat_template, batched=True)

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

In [17]:
from trl import SFTTrainer, SFTConfig
trainer=SFTTrainer(
        model=model,
        train_dataset=hf_dataset,
        max_seq_length=2048,
        tokenizer=tokenizer,
        eval_dataset=None,
        args = SFTConfig(
            dataset_text_field = "text",
            per_device_train_batch_size = 2,
            per_device_eval_batch_size = 4,
            warmup_steps = 5,
            max_steps= 30,
            learning_rate=2e-4,
            logging_steps =1,
            optim = 'adamw_8bit',
            weight_decay = 0.01,
            lr_scheduler_type = 'linear',
            seed= 3407,
            report_to = 'none',
            dataset_num_proc= 2
        )
)
#         train_dataset=hf_dataset,
#         max_seq_length=2048,
#         tokenizer=tokenizer,
#         eval_dataset=None,
#         args = SFTConfig(
#             dataset_text_feild = "text",
#             per_device_train_batch_size = 2,
#             per_device_eval_batch_size = 4,
#             warmup_step = 5,
#             max_steps= 30,
#             learning_rate=2e-4,
#             logging_steps =1,
#             optim = 'adam_8bit',
#             weigh_decay = 0.01,
#             lr_scheduler_type = 'linear',
#             seed= 3407,
#             report_to = 'none'
#         )
# )

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/90 [00:00<?, ? examples/s]

In [18]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part="<start_of_turn>model\n"
)

Map (num_proc=2):   0%|          | 0/90 [00:00<?, ? examples/s]

In [19]:

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 90 | Num Epochs = 2 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 19,248,896/4,000,000,000 (0.48% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.952400
2,4.680700
3,4.922100
4,7.382200
5,5.372200
6,5.569300
7,3.732000
8,3.750600
9,3.473800
10,4.029100


In [20]:
# Save the entire model and tokenizer locally
model.save_pretrained("gemma-4b-offline")
tokenizer.save_pretrained("gemma-4b-offline")

['gemma-4b-offline/processor_config.json']

In [23]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/gemma-4b-offline",  # Local folder path
    max_seq_length = 2048,
    dtype = torch.float16, # Changed "float16" to torch.float16
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


In [24]:
FastLanguageModel.for_inference(model)

prompt = """<|user|>\nHello, may I see the menu?\n<|assistant|>"""
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


<|user|>
Hello, may I see the menu?
<|assistant|>
Okay, here's our menu! 

**Appetizers**

*   **Bruschetta:** Toasted baguette slices topped with fresh tomatoes, basil, garlic, and balsamic glaze - $8
*   **Spinach and Artichoke Dip:** Creamy spinach and artichoke dip served with warm pita bread - $10
*   **Fried Calamari:** Lightly battered and fried calamari served with marinara sauce - $12
*   **


In [26]:
!zip -r gemma-4b-offline.zip gemma-4b-offline/
# Transfer via USB or shared storage

  adding: gemma-4b-offline/ (stored 0%)
  adding: gemma-4b-offline/tokenizer.model (deflated 52%)
  adding: gemma-4b-offline/chat_template.json (deflated 70%)
  adding: gemma-4b-offline/tokenizer_config.json (deflated 96%)
  adding: gemma-4b-offline/processor_config.json (deflated 11%)
  adding: gemma-4b-offline/preprocessor_config.json (deflated 55%)
  adding: gemma-4b-offline/README.md (deflated 66%)
  adding: gemma-4b-offline/adapter_config.json (deflated 57%)
  adding: gemma-4b-offline/adapter_model.safetensors (deflated 23%)
  adding: gemma-4b-offline/added_tokens.json (stored 0%)
  adding: gemma-4b-offline/special_tokens_map.json (deflated 77%)
  adding: gemma-4b-offline/tokenizer.json (deflated 83%)


In [27]:
!git lfs install
!git clone https://huggingface.co/google/gemma-4b-it


Git LFS initialized.
Cloning into 'gemma-4b-it'...
fatal: could not read Username for 'https://huggingface.co': No such device or address


In [30]:
from peft import AutoPeftModelForCausalLM

# Load base + adapter model
model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/gemma-4b-offline",  # Path to your fine-tuned model folder (with adapter)
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
).merge_and_unload()  # <-- This merges LoRA into base

# Save full merged model
model.save_pretrained("gemma-4b-full-merged", safe_serialization=True)
tokenizer.save_pretrained("gemma-4b-full-merged")


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


['gemma-4b-full-merged/processor_config.json']

In [31]:
from google.colab import files
import os
import shutil

# Create a zip archive of the folder
shutil.make_archive('gemma-4b-full-merged', 'zip', '/content/gemma-4b-full-merged')

# Download the zip archive
files.download('gemma-4b-full-merged.zip')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>